In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim

from src.models import CifarResNet, MNIST_CNN, CIFAR_CNN
from src.helpers import evaluate_rob_accuracy, evaluate_clean_accuracy, load_model, safe_model,_evaluate_model
from src.data_loader import load_torchvision_dataset, load_imagenette
#from src.pruning import identify_layers, _evaluate_sparsity

import time

if torch.cuda.is_available() == True:
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")
print(device)
dtype = torch.float32

cuda:0


In [2]:
model = CifarResNet()
model.to(device)

identifying layers


CifarResNet(
  (c1): MaskedConvLayer()
  (r1): ResBlock(
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (c1): MaskedConvLayer()
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (c2): MaskedConvLayer()
  )
  (r2): ResBlock(
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (c1): MaskedConvLayer()
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (c2): MaskedConvLayer()
  )
  (r3): ResBlock(
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (c1): MaskedConvLayer()
    (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (c2): MaskedConvLayer()
    (c3): MaskedConvLayer()
  )
  (r4): ResBlock(
    (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (c1): MaskedConvLayer()
    (bn2): Bat

In [3]:
train_loader, test_loader = load_torchvision_dataset('CIFAR10', data_augmentation=True)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
train_stats = model.fit(train_loader, test_loader, 1, device)

[1,     1] loss: 1.96118, train_accuracy: 29.88
[1,     2] loss: 5.29874, train_accuracy: 19.73
[1,     3] loss: 5.81986, train_accuracy: 9.96
[1,     4] loss: 3.74798, train_accuracy: 10.94
[1,     5] loss: 3.13215, train_accuracy: 19.34
[1,     6] loss: 3.23406, train_accuracy: 16.21
[1,     7] loss: 2.71220, train_accuracy: 14.06
[1,     8] loss: 2.70970, train_accuracy: 12.50
[1,     9] loss: 2.92613, train_accuracy: 16.21
[1,    10] loss: 3.54328, train_accuracy: 20.70
[1,    11] loss: 2.51670, train_accuracy: 19.73
[1,    12] loss: 3.60842, train_accuracy: 18.36
[1,    13] loss: 2.74857, train_accuracy: 21.48
[1,    14] loss: 3.33330, train_accuracy: 19.34
[1,    15] loss: 2.32501, train_accuracy: 17.97
[1,    16] loss: 2.24358, train_accuracy: 22.27
[1,    17] loss: 2.64021, train_accuracy: 19.73
[1,    18] loss: 2.76872, train_accuracy: 21.48
[1,    19] loss: 2.12910, train_accuracy: 23.63
[1,    20] loss: 2.65200, train_accuracy: 22.07
[1,    21] loss: 2.74562, train_accuracy:

In [13]:
model.train_stats['l_inf_robustness'] = model.train_stats['l_inf_robustness'].map(lambda x: x.item())
model.train_stats['l_2_robustness'] = model.train_stats['l_2_robustness'].map(lambda x: x.item())

In [14]:
model.train_stats

,epoch,train_loss,train_accuracy,validation_loss,l_inf_robustness,l_inf_loss,l_2_robustness,l_2_loss,l_0_robustness,l_0_loss,validation_accuracy,duration,criterion,optimizer,method,learning_rate,batchsize,test
0,1,4.493002,20.343534,2.253086,0.279297,NaN,0.289062,NaN,0,NaN,28.59,112.124355,CrossEntropyLoss(),Adam (\nParameter Group 0\n amsgrad: False\...,standard,NaN,336,0.279297
1,2,2.304893,25.700202,1.882308,0.279297,NaN,0.289062,NaN,0,NaN,33.85,225.014890,CrossEntropyLoss(),Adam (\nParameter Group 0\n amsgrad: False\...,standard,NaN,336,0.279297
2,1,2.316611,24.480970,1.760163,0.337891,NaN,0.355469,NaN,0,NaN,35.15,111.586921,CrossEntropyLoss(),Adam (\nParameter Group 0\n amsgrad: False\...,standard,NaN,336,0.337891


In [ ]:
PATH = './saved-models/cifar-resnet-100e-data_aug'
safe_model(PATH, model, optimizer_state_dict, description='w/ full data auggmentation pipeline', loss='N/A',epoch='100')

In [ ]:
PATH = './saved-models/cifar-resnet-100e-data_aug'
model = load_model(model, PATH)

In [ ]:
model.train_stats.plot(x='epoch', y=['validation_loss','validation_accuracy'])

In [ ]:
model.train_stats.plot(x='epoch', y=['train_loss','train_accuracy'])